In [4]:
import numpy as np
import scipy
from scipy import stats
from scipy.stats import ttest_ind

### Реализовать t-критерий Стьюдента для 2 независимых выборок при условии неизвестной дисперсии. Необходимо считать значение статистики и p_value для разных видов гипотез (двусторонняя, односторонние), а также результат (отвергается нулевая гипотеза или нет). Сравнить результаты с реализацией в scipy.stats.ttest_ind.

In [207]:
shapes = (10000, 10000)
mu = 10
sigma1 = 10
sigma2 = 50

In [208]:
def experiment(arr_1, arr_2, var_eq=True, two_sided=True, alpha=0.05):
    means = [np.mean(arr) for arr in [arr_1, arr_2]]
    n = len(arr_1)
    m = len(arr_2)
    s = [np.std(arr, ddof=1) ** 2 for arr in [arr_1, arr_2]]
    
    if var_eq:
        s2 = (s[0] * (n - 1) + s[1] * (m - 1)) / (m + n - 2)
        t = (means[0] - means[1]) / np.sqrt(s2) * np.sqrt(n * m / (n + m))
        dim = n + m - 2
    else:
        s2 = (s[0] / n) + (s[1] / m)
        t = (means[0] - means[1]) / np.sqrt(s2)
        dim = s2 ** 2 / ((s[0] / n) ** 2 / (n - 1) + (s[1] / m) ** 2 / (m - 1)) - 2
    
    fst = 2 * (stats.t.cdf(abs(t), dim))
    sec = 2 * (1 - stats.t.cdf(abs(t), dim))
    p_value = min(fst, sec)
    
    if not two_sided:
        if means[0] > means[1]:
            p_value = p_value / 2
        else:
            p_value = 1.0 - p_value / 2.

    if p_value <= alpha:
        print(f"{round(p_value, 3)} <= {alpha}", 'принимаем нулевую гипотезу')

    else:
        print(f"{round(p_value, 3)} > {alpha}", 'отвергаем нулевую гипотезу')
    
    return t, p_value

In [209]:
print('Двусторонний критерий')
print('\nПри равенстве дисперсий\n')
arrs_1 = stats.norm.rvs(loc=mu, scale=sigma1, size=shapes[0])
arrs_2 = stats.norm.rvs(loc=mu, scale=sigma1, size=shapes[1])
print("реализация stats:\n", stats.ttest_ind(arrs_1, arrs_2, equal_var=True, alternative='two-sided'))
res = experiment(arrs_1, arrs_2,  var_eq=True, two_sided=True)
print(f"t {res[0]}\np_value:{res[1]}")

print('\nПри различных дисперсиях\n')
arrs_1 = stats.norm.rvs(loc=mu, scale=sigma1, size=shapes[0])
arrs_2 = stats.norm.rvs(loc=mu, scale=sigma2, size=shapes[1])
print("реализация stats:\n", stats.ttest_ind(arrs_1, arrs_2,  equal_var=True, alternative='two-sided'))
res = experiment(arrs_1, arrs_2,  var_eq=False, two_sided=True)
print(f"t {res[0]}\np_value:{res[1]}")



Двусторонний критерий

При равенстве дисперсий

реализация stats:
 Ttest_indResult(statistic=-1.1338726053954962, pvalue=0.25686158183640445)
0.257 > 0.05 отвергаем нулевую гипотезу
t -1.1338726053954962
p_value:0.2568615818364044

При различных дисперсиях

реализация stats:
 Ttest_indResult(statistic=-0.42173701396836616, pvalue=0.6732215175586687)
0.673 > 0.05 отвергаем нулевую гипотезу
t -0.42173701396836616
p_value:0.6732253842222371


In [210]:
print('Односторонний критерий')
print('\nПри равенстве дисперсий\n')
arrs_1 = stats.norm.rvs(loc=mu, scale=sigma1, size=shapes[0])
arrs_2 = stats.norm.rvs(loc=mu, scale=sigma1, size=shapes[1])
print("реализация stats:\n", stats.ttest_ind(arrs_1, arrs_2, equal_var=True, alternative='greater'))
res = experiment(arrs_1, arrs_2, var_eq=True, two_sided=False)
print(f"t {res[0]}\np_value:{res[1]}")

print('\nПри различных дисперсиях\n')
arrs_1 = stats.norm.rvs(loc=mu, scale=sigma1, size=shapes[0])
arrs_2 = stats.norm.rvs(loc=mu, scale=sigma2, size=shapes[1])
print("реализация stats:\n", stats.ttest_ind(arrs_1, arrs_2,  equal_var=False, alternative='greater'))
res = experiment(arrs_1, arrs_2, var_eq=False, two_sided=False)
print(f"t {res[0]}\np_value:{res[1]}")

Односторонний критерий

При равенстве дисперсий

реализация stats:
 Ttest_indResult(statistic=-1.5680641273211138, pvalue=0.9415590054545164)
0.942 > 0.05 отвергаем нулевую гипотезу
t -1.568064127321114
p_value:0.9415590054545164

При различных дисперсиях

реализация stats:
 Ttest_indResult(statistic=-0.4810961803991596, pvalue=0.6847710476031972)
0.685 > 0.05 отвергаем нулевую гипотезу
t -0.4810961803991596
p_value:0.6847710466976695


Вывод: собственная реализация не отличается от библиотечной,

### Реализовать bootstrap для оценки (через доверительные интервалы) медианных и средних значений распределений. Рассчитать для распределений: нормальное, экспоненциальное, смесь нормальных

In [148]:
def bootstrap(arr, f, m=10000, alpha=0.05):
    values = []
    n = len(arr)

    for i in range(m):
        subsample = arr[np.random.randint(0, n, n)]
        values.append(f(subsample))

    return np.percentile(values, (alpha * 100, (1 - alpha) * 100))

In [179]:
sample_size = 10000

In [164]:
loc = 5
scale = 1
print('Нормальное распределение')
arr_norm = stats.norm.rvs(loc=loc, scale=scale, size=sample_size)
print('Матожидание')
print(f"Истинное значение {loc}")
print(f"Интервальная оценка {bootstrap(arr_norm, np.mean)}")
print('\nМедиана')
print(f"Истинное значение {loc}")
print(f"Интервальная оценка {bootstrap(arr_norm, np.median)}")

Нормальное распределение
Матожидание
Истинное значение 5
Интервальная оценка [4.97804368 5.01141598]

Медиана
Истинное значение 5
Интервальная оценка [4.9586902  4.99678536]


In [169]:
scale = 0.1
loc = 0
print('Экспоненциальное распределение')
arr_exp = stats.expon.rvs(loc=loc, scale=scale, size=sample_size)
print('Матожидание')
print(f"Истинное значение {scale}")
print(f"Интервальная оценка {bootstrap(arr_exp, np.mean)}")
print('\nМедиана')
print(f"Истинное значение {np.log(2) * scale}")
print(f"Интервальная оценка {bootstrap(arr_exp, np.median)}")

Экспоненциальное распределение
Матожидание
Истинное значение 0.1
Интервальная оценка [0.09740875 0.10066228]

Медиана
Истинное значение 0.06931471805599453
Интервальная оценка [0.06731385 0.07042109]


In [171]:
w = np.array([0.1, 0.4, 0.5])
locs = np.array([0, 2, 3])
scales = np.array([1, 3, 2])

In [201]:
mixture = [0] * sample_size
for i in range(len(w)):
    mixture +=  w[i] * stats.norm.rvs(loc=locs[i], scale=scales[i], size=sample_size) 

real_mean = np.sum(np.multiply(w, locs))

print('Смесь нормальных распределений')
print('Матожидание')
print(f"Истинное значение {real_mean}")
print(f"Интервальная оценка {bootstrap(mixture, np.mean)}")
print('\nМедиана')
print(f"Истинное значение {real_mean}")
print(f"Интервальная оценка {bootstrap(mixture, np.median)}")

Смесь нормальных распределений
Матожидание
Истинное значение 2.3
Интервальная оценка [2.2989973  2.35012118]

Медиана
Истинное значение 2.3
Интервальная оценка [2.29215714 2.36280353]


Вывод: интервальные оценки соответствуют истинным значениям

### Рассчитать мощность критерия для t-критерия и критерия Манна-Уитни для разных распределений и разном эффекте. Отдельно изучить случай, когда средние равны, а дисперсии сильно отличаются.

Нормальное распределение

In [25]:
exps = 1000

In [2]:
def experiment(rvs, criteria, loc=0, scale=1, diff=0, scale_diff=0, size=10000, alpha=0.05, exps=1000):
    p_values = []
    for i in range(exps):
        arr_1 = rvs(loc=loc, scale=scale, size=size)
        arr_2 = rvs(loc=loc + diff , scale=scale + scale_diff, size=size)
        res = criteria(arr_1, arr_2, alternative='two-sided')
        p_values.append(res.pvalue)

    p_values = np.array(p_values)

    mask = p_values < alpha
    power = p_values[mask].shape[0] / exps

    return p_values, power

In [252]:
for diff in (0.01, 0.05, 0.1):
    for scale_diff in (0, 0.1, 0.2):
        _, power = experiment(stats.norm.rvs, stats.ttest_ind, diff=diff, scale_diff=scale_diff)
        print(f" Мощность T-теста при loc2= {diff} + loc2 и scale2= {scale_diff} + scale1", power, sep=': ')

        _, power = experiment(stats.norm.rvs, stats.mannwhitneyu, diff=diff, scale_diff=scale_diff)
        print(f" Мощность Манна-Уитни при loc2= {diff} + loc2 и scale2= {scale_diff} + scale1", power, sep=': ')

        print('-' * 30)

 Мощность T-теста при loc2= 0.01 + loc2 и scale2= 0 + scale1: 0.112
 Мощность Манна-Уитни при loc2= 0.01 + loc2 и scale2= 0 + scale1: 0.101
------------------------------
 Мощность T-теста при loc2= 0.01 + loc2 и scale2= 0.1 + scale1: 0.098
 Мощность Манна-Уитни при loc2= 0.01 + loc2 и scale2= 0.1 + scale1: 0.109
------------------------------
 Мощность T-теста при loc2= 0.01 + loc2 и scale2= 0.2 + scale1: 0.091
 Мощность Манна-Уитни при loc2= 0.01 + loc2 и scale2= 0.2 + scale1: 0.096
------------------------------
 Мощность T-теста при loc2= 0.05 + loc2 и scale2= 0 + scale1: 0.947
 Мощность Манна-Уитни при loc2= 0.05 + loc2 и scale2= 0 + scale1: 0.932
------------------------------
 Мощность T-теста при loc2= 0.05 + loc2 и scale2= 0.1 + scale1: 0.913
 Мощность Манна-Уитни при loc2= 0.05 + loc2 и scale2= 0.1 + scale1: 0.911
------------------------------
 Мощность T-теста при loc2= 0.05 + loc2 и scale2= 0.2 + scale1: 0.891
 Мощность Манна-Уитни при loc2= 0.05 + loc2 и scale2= 0.2 + sca

Экспоненциальное распределение

In [5]:
for diff in (0.01, 0.05, 0.1):
    for scale_diff in (0, 0.1, 0.2):
        _, power = experiment(stats.expon.rvs, stats.ttest_ind, diff=diff, scale_diff=scale_diff)
        print(f" Мощность T-теста при loc2= {diff} + loc2 и scale2= {scale_diff} + scale1", power, sep=': ')

        _, power = experiment(stats.expon.rvs, stats.mannwhitneyu, diff=diff, scale_diff=scale_diff)
        print(f" Мощность Манна-Уитни при loc2= {diff} + loc2 и scale2= {scale_diff} + scale1", power, sep=': ')

        print('-' * 30)

 Мощность T-теста при loc2= 0.01 + loc2 и scale2= 0 + scale1: 0.114
 Мощность Манна-Уитни при loc2= 0.01 + loc2 и scale2= 0 + scale1: 0.252
------------------------------
 Мощность T-теста при loc2= 0.01 + loc2 и scale2= 0.1 + scale1: 1.0
 Мощность Манна-Уитни при loc2= 0.01 + loc2 и scale2= 0.1 + scale1: 1.0
------------------------------
 Мощность T-теста при loc2= 0.01 + loc2 и scale2= 0.2 + scale1: 1.0
 Мощность Манна-Уитни при loc2= 0.01 + loc2 и scale2= 0.2 + scale1: 1.0
------------------------------
 Мощность T-теста при loc2= 0.05 + loc2 и scale2= 0 + scale1: 0.944
 Мощность Манна-Уитни при loc2= 0.05 + loc2 и scale2= 0 + scale1: 1.0
------------------------------
 Мощность T-теста при loc2= 0.05 + loc2 и scale2= 0.1 + scale1: 1.0
 Мощность Манна-Уитни при loc2= 0.05 + loc2 и scale2= 0.1 + scale1: 1.0
------------------------------
 Мощность T-теста при loc2= 0.05 + loc2 и scale2= 0.2 + scale1: 1.0
 Мощность Манна-Уитни при loc2= 0.05 + loc2 и scale2= 0.2 + scale1: 1.0
-------

### Оценить корректность t-критерия и критерия Манна-Уитни на разных распределениях.

In [6]:
import plotly.graph_objects as go

: 

In [244]:
def plot_qq(rvs, criteria):
    p_values, _ = experiment(rvs, criteria)
    y = []
    x = np.linspace(0.01, 1, 100)
    for a in x:
        y.append(p_values[p_values < a].shape[0] / p_values.shape[0])

    fig = go.Figure([go.Scatter(x=x, y=y, mode="markers", name = "p_value"),
                    go.Scatter(x=x,y=x, mode="lines",name="uniform")])
    fig.update_layout(height=600, width=600, title="Q-Q plot")
    return fig

T-test, нормальное распределение

In [245]:
plot_qq(stats.norm.rvs, stats.ttest_ind)

Манн-Уитни, нормальное распределение

In [251]:
plot_qq(stats.norm.rvs, stats.mannwhitneyu)

Манн-Уитни, экспоненциальное распределение

In [240]:
plot_qq(stats.expon.rvs, stats.mannwhitneyu)


T-test, экспоненциальное распределение

In [249]:
plot_qq(stats.expon.rvs, stats.ttest_ind)

Вывод: все критерии корректны